# Wetland Change in the Chaco

## Part 01: Create composites in Google Earth Engine and download

##### (c) Matthias Baumann (with strong support by Julian Oeser)
Initial script creation: 2018-08-03

In [1]:
# Import all other packages needed
import os
import re
import baumiTools as bt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import time

In [2]:
# Import and initialize Google Earth Engine
import ee
ee.Initialize()

### This script creates annual landsat composites in Google Earth Engine and downloads them onto a local disc.
##### Step (1): Manually upload tile-scheme as feature collection into GEE, and load here as asset
* Load as variable 'tiles'
* the id-Field to be accessed is 'ID' or 'TileIndex

In [3]:
tiles = ee.FeatureCollection('users/matthiasbaumann84/CHACO_tiles_v02')
tilevar = 'Id'
tile_names = tiles.distinct(tilevar).aggregate_array(tilevar).getInfo()
features = ee.Feature(tiles.first()).propertyNames().sort()#.remove(response).remove('system:index').remove(tilevar)

In [4]:
localFolder = 'Z:/CHACO/_LANDSAT/annual_Metrics/1991/'

##### Step (2): Collect all Landsat T1 surface reflectance data for a given region and time period, calculate spectral indices.
* apply masking (e.g., clouds) based on Pixel-QA band, 
* remove double observations from WRS path overlap areas,
* harmonize Landsat 8 reflectance with Landsat 7 based on coefficients from Roy et al. 2016 (https://www.sciencedirect.com/science/article/pii/S0034425715302455, Table 2)

In [5]:
def landsat_collect (roi, 
                     start, 
                     end, 
                     seasonal_filter = ee.Filter.dayOfYear(1, 366),
                     mask_clouds = True,
                     mask_water = False,
                     mask_snow = False,
                     mask_fill = True,
                     harmonize_l8 = True):
    
    select_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
    # band names in input and output collections
    bands = ['B1', 'B2', 'B3', 'B4', 'B5','B7', 'B6', 'pixel_qa']
    band_names = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'T','pixel_qa']
    l8bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B1', 'B10', 'B11', 'pixel_qa']
    l8band_names = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'UB', 'T1', 'T2','pixel_qa']

    # qa bits 
    cloudbit = ee.Number(ee.Algorithms.If(mask_clouds, 40, 0))
    waterbit = ee.Number(ee.Algorithms.If(mask_water, 4, 0))
    snowbit = ee.Number(ee.Algorithms.If(mask_snow, 16, 0))
    fillbit = ee.Number(ee.Algorithms.If(mask_fill, 1, 0))
    bits = cloudbit.add(waterbit).add(snowbit).add(fillbit)

    ## helper functions
    # function to apply masks based on pixel qa band
    def apply_masks (img):
        qa = img.select('pixel_qa')
        mask = qa.bitwiseAnd(bits).eq(0)
        return img.updateMask(mask)

    # function to harmonize l8 surface reflectance with coefficients from Roy et al. 2016
    def l8_harmonize (l8img):
        b = ee.Image(0.0183).add(ee.Image(0.8850).multiply(l8img.select('B'))).int16()
        g = ee.Image(0.0123).add(ee.Image(0.9317).multiply(l8img.select('G'))).int16()
        r = ee.Image(0.0123).add(ee.Image(0.9372).multiply(l8img.select('R'))).int16()
        nir = ee.Image(0.0448).add(ee.Image(0.8339).multiply(l8img.select('NIR'))).int16()
        swir1 = ee.Image(0.0306).add(ee.Image(0.8639).multiply(l8img.select('SWIR1'))).int16()
        swir2 = ee.Image(0.0116).add(ee.Image(0.9165).multiply(l8img.select('SWIR2'))).int16()

        out = ee.Image(b.addBands(g).addBands(r).addBands(nir).addBands(swir1).addBands(swir2).addBands(l8img.select(['UB', 'T1', 'T2','pixel_qa'])).copyProperties(l8img, l8img.propertyNames())).rename(l8band_names)
        return out
    
    # function to remove double counts from path overlap areas
    def remove_double_counts (collection):
        
        def add_nn(image):
            start = ee.Date.fromYMD(image.date().get('year'), image.date().get('month'), image.date().get('day')).update(hour = 0, minute = 0, second = 0)
            end = ee.Date.fromYMD(image.date().get('year'), image.date().get('month'), image.date().get('day')).update(hour = 23, minute = 59, second = 59)
            overlapping = collection.filterDate(start, end).filterBounds(image.geometry())
            nn = overlapping.filterMetadata('WRS_ROW', 'equals', ee.Number(image.get('WRS_ROW')).subtract(1)).size()
            return image.set('nn', nn)
        
        collection_nn = collection.map(add_nn)
        has_nn = collection_nn.filterMetadata('nn', 'greater_than', 0)
        has_no_nn = ee.ImageCollection(ee.Join.inverted().apply(collection, has_nn, ee.Filter.equals(leftField = 'LANDSAT_ID', rightField = 'LANDSAT_ID')))
      
        def mask_overlap(image):
            start = ee.Date.fromYMD(image.date().get('year'), image.date().get('month'), image.date().get('day')).update(hour = 0, minute = 0, second = 0)
            end = ee.Date.fromYMD(image.date().get('year'), image.date().get('month'), image.date().get('day')).update(hour = 23, minute = 59, second = 59)
            overlapping = collection.filterDate(start, end).filterBounds(image.geometry())
            nn = ee.Image(overlapping.filterMetadata('WRS_ROW', 'equals', ee.Number(image.get('WRS_ROW')).subtract(1)).first())
            newmask = image.mask().where(nn.mask(), 0)
            return image.updateMask(newmask)

        has_nn_masked = ee.ImageCollection(has_nn.map(mask_overlap))
        out = ee.ImageCollection(has_nn_masked.merge(has_no_nn).copyProperties(collection))
        return out
  
    # get landsat data, apply filters and masks
    l4 = remove_double_counts(ee.ImageCollection('LANDSAT/LT04/C01/T1_SR').select(bands, band_names).filterBounds(roi).filterDate(start, end).filter(seasonal_filter).map(apply_masks))
    l5 = remove_double_counts(ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').select(bands, band_names).filterBounds(roi).filterDate(start, end).filter(seasonal_filter).map(apply_masks))
    l7 = remove_double_counts(ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').select(bands, band_names).filterBounds(roi).filterDate(start, end).filter(seasonal_filter).map(apply_masks))
    l8 = remove_double_counts(ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').select(l8bands, l8band_names).filterBounds(roi).filterDate(start, end).filter(seasonal_filter).map(apply_masks))
    l8h = ee.ImageCollection(ee.Algorithms.If(harmonize_l8, l8.map(l8_harmonize), l8))

    # combine landsat collections
    landsat = ee.ImageCollection(l4.merge(l5).merge(l7).merge(l8h)).select(select_bands)
    return landsat

* calculate  some additional spectral indices for the classification
* For the tasseled cap transformation, also 'forurth', 'fifth' and 'sixth' component are calculated. for the download of the tile lateron not considered, though. can be done by commenting the resptive line

In [6]:
def landsat_indices (landsat, indices):
    
    # helper functions to add spectral indices to collections
    def add_NBR (image):
        nbr = image.normalizedDifference(['NIR', 'SWIR2']).rename('NBR')
        return image.addBands(nbr)
    def add_NDMI (image):
        ndvi = image.normalizedDifference(['NIR', 'SWIR1']).rename('NDMI')
        return image.addBands(ndvi)
    def add_EVI (image):
        evi = image.expression('2.5 * ((NIR - R) / (NIR + 6 * R - 7.5 * B + 1))', {'NIR': image.select('NIR'),'R': image.select('R'),'B': image.select('B')}).rename('EVI')
        return image.addBands(evi)
    def add_MSAVI (image): 
        msavi = image.expression('(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - R)) ) / 2', {'NIR': image.select('NIR'),'R': image.select('R')}).rename('MSAVI')
        return image.addBands(msavi)
    def add_TC (image):
        
        img = image.select(['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2'])
        # coefficients for Landsat surface reflectance (Crist 1985)
        brightness_c= ee.Image([0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863])
        greenness_c= ee.Image([-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800])
        wetness_c= ee.Image([0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572])
        fourth_c= ee.Image([-0.8242, 0.0849, 0.4392, -0.0580, 0.2012, -0.2768])
        fifth_c= ee.Image([-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085])
        sixth_c= ee.Image([0.1084, -0.9022, 0.4120, 0.0573, -0.0251, 0.0238])

        brightness = img.multiply(brightness_c)
        greenness = img.multiply(greenness_c)
        wetness = img.multiply(wetness_c)
        fourth = img.multiply(fourth_c)
        fifth = img.multiply(fifth_c)
        sixth = img.multiply(sixth_c)

        brightness = brightness.reduce(ee.call('Reducer.sum'))
        greenness = greenness.reduce(ee.call('Reducer.sum'))
        wetness = wetness.reduce(ee.call('Reducer.sum'))
        fourth = fourth.reduce(ee.call('Reducer.sum'))
        fifth = fifth.reduce(ee.call('Reducer.sum'))
        sixth = sixth.reduce(ee.call('Reducer.sum'))

        tasseled_cap = ee.Image(brightness).addBands(greenness).addBands(wetness).rename(['brightness','greenness','wetness'])
        #tasseled_cap = ee.Image(brightness).addBands(greenness).addBands(wetness).addBands(fourth).addBands(fifth).addBands(sixth)rename(['brightness','greenness','wetness', 'fourth', 'fifth', 'sixth']
        return image.addBands(tasseled_cap)

    out = landsat.map(add_NBR).map(add_NDMI).map(add_EVI).map(add_MSAVI).map(add_TC).select(indices)
    return out

##### Step (3): Create composites
* Build some helper functions.
    * To stack the images and collections, when iterating over years.

In [7]:
def stack_images(image_list): 
    first = ee.Image(image_list.get(0)).select([])
    
    def add_img (image, previous):
        return ee.Image(previous).addBands(image)
    
    return ee.Image(image_list.iterate(add_img, first))


def stack_collection(collection):
    first = ee.Image(collection.first()).select([])
    
    def add_img (image, previous):
        return ee.Image(previous).addBands(image)
    
    return ee.Image(collection.iterate(add_img, first))

* Now, we build the function to generate the predictors per tile
    * Mean, sd, median, 10/25/75/90 percentiles (in that exact order) of the six multispectral bands
    * Mean, sd, median, 10/25/75/90 percentiles (in that exact order) of NBR, NDMI, EVI, MSAVI, TC_B, TC_G, TC_W
* In the example below this can be done by determining a sequence of years (startYear/endYear)
    * For the current project (i.e., downloading composites for large tiles) startYear = endYear
    * Can be adjusted according to the needs
* One can also decide which predictors to include
    + Landsat spectral metrics only
    + Index metrics only
    + Landsat and index metrics

In [8]:
def calculate_predictors_all(tile):
    # define start year and end year for the compositing
    startYear = 1991
    endYear = 1991
    years = ee.List.sequence(startYear, endYear)
    
    # Define some start specific values
    roi = tiles.filterMetadata(tilevar, 'equals', tile)  
    left = ee.Number(1)
    right = ee.Number(1)
    
    def year_predictors(year):
        # Define seasonal window --> here: entire year
        start = ee.Date.fromYMD(ee.Number(year).subtract(ee.Number(1)), 1, 1)
        end = ee.Date.fromYMD(ee.Number(year).add(ee.Number(1)), 12, 31)
        start_window = ee.Date.fromYMD(ee.Number(year).subtract(left), 1, 1)
        end_window = ee.Date.fromYMD(ee.Number(year).add(right), 12, 31)

        # Collect the data
        landsat_data = landsat_collect(roi, start_window, end_window).select(['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2'])
        spectral_indices = landsat_indices(landsat_data, ["NBR", "NDMI", "EVI", "MSAVI", "wetness", "brightness", "greenness"])
        
        # Create the reducers
        mean = ee.Reducer.mean().unweighted()
        sd = ee.Reducer.stdDev().unweighted()
        median = ee.Reducer.percentile([50]). unweighted()
        percentiles = ee.Reducer.percentile([10,25,75,90]).unweighted()
        #minmax = ee.Reducer.minMax().unweighted()
        metrics_summary = mean.combine(sd, sharedInputs = True).combine(median, sharedInputs = True).combine(percentiles, sharedInputs= True)
        
        # Calculate the predictors
        #predictors = landsat_data.reduce(metrics_summary)
        #predictors = spectral_indices.reduce(metrics_summary)
        predictors = landsat_data.reduce(metrics_summary).addBands(spectral_indices.reduce(metrics_summary))
        return predictors.set('system:time_start', start.millis())
    
    predictor_year = years.map(year_predictors)
    return ee.ImageCollection(predictor_year)

* Helper-function to rename bands to short names. Needed, because if you use the full band names, you run into trouble when creating a .shp out of the exports (maximum variable name length)
* not elegant, but the easiest workaround that i found

In [9]:
def rename_bands(multiband):
    old_bandnames = multiband.bandNames()
    bandseq = ee.List.sequence(1, old_bandnames.size())
    def create_bandnames(i):
        return ee.String('v').cat(ee.Number(i).format('%03d'))
    new_bandnames = bandseq.map(create_bandnames)
    return multiband.select(old_bandnames, new_bandnames)

##### Step (4): Export the composites to disc via google-drive
* Build export-function.
    * Convert values into integer values to safe space
        * In case of using only Landsat metrics --> convert to int only
        * In case of using Landsat and/or index metrics --> convert to int and multiply by 10000 (needs to be updated)
    * Define folder, that you previously generated on your Google Drive
        * Here: 'Baumi_GEE'
* Check locally for existing processed tiles
    * Cell needs to be re-run each time before starting the export. Otherwise, tiles are getting re-processed
* Authenticate your google account
    * infos on how to create the *client_secrets.py* file here: *https://pythonhosted.org/PyDrive/quickstart.html#authentication*
    * then copy the file into the folder from where we execute the script
* start the batch download to sequentially download the composites

In [10]:
def export_composite(tile):
  
    roi = tiles.filterMetadata(tilevar, 'equals', tile)
    #To safe space, multiply floating values by 10000
    #tile_collection = stack_collection(calculate_predictors_all(tile).map(rename_bands)).int()
    tile_collection = stack_collection(calculate_predictors_all(tile).map(rename_bands)).multiply(10000).int()

    description = 'export_' + str(tile)
    fileNamePrefix = 'tileID_' + str(tile)
    GEE_Folder = 'Baumi_GEE'
    
    task= ee.batch.Export.image.toDrive(
        image = tile_collection,
        description = description,
        folder = GEE_Folder,
        fileNamePrefix = fileNamePrefix,
        region = roi.geometry().getInfo()['coordinates'],
        scale = 30)
    
    task.start()

In [11]:
file_list =[]
for file in os.listdir(localFolder):
    if file.endswith(".tif"):
        file_list.append(file)
processed_tiles = []
for filename in file_list:
    result = re.search('\d+', filename)
    processed_tiles.append(result.group(0))
missing_tiles = []
for tile in tile_names:
    t = str(tile)
    if(t not in processed_tiles):
        missing_tiles.append(t)

In [12]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=9291758365-jcrgrd0a3gfv10ipqfc5vq675ua5hve4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


Run the actual generation and export of tiles
* Important: copy the ID of your google-folder as a string into the the variable 'drive_list' (is highlighted slightly)

In [13]:
n_tasks = 0

for tile in missing_tiles:
    while(n_tasks >=15):
        time.sleep(60)
        try:
            task_list = str(ee.batch.Task.list())
            n_running = task_list.count('RUNNING')
            n_ready = task_list.count('READY')
            n_tasks = n_running + n_ready
        except:
            time.sleep(5)

    export_composite(int(tile))
    task_list = str(ee.batch.Task.list())
    n_running = task_list.count('RUNNING')
    n_ready = task_list.count('READY')
    n_tasks = n_running + n_ready
    #
    drive_list = drive.ListFile({'q': "'0B9hZR9DKK3xRS3phZnhQcVlwVHc' in parents and trashed=false"}).GetList()
    #
    for file in drive_list:
        file_id = drive.CreateFile({'id': file['id']})
        fname = file["title"]
        file_id.GetContentFile(localFolder + fname)
        file_id.Delete() 

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=9291758365-jcrgrd0a3gfv10ipqfc5vq675ua5hve4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=9291758365-jcrgrd0a3gfv10ipqfc5vq675ua5hve4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=9291758365-jcrgrd0a3gfv10ipqfc5vq675ua5hve4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful